In [1]:
import cv2
import numpy as np

In [5]:
background_image= cv2.imread("background_image.PNG")
gray_background = cv2.cvtColor(background_image, cv2.COLOR_BGR2GRAY)
height, width, _ = background_image.shape
ball_radius=35
ball_color =(0,0,0)

ball_x=width // 2
ball_y=231
speed=10
velocity_x= speed
velocity_y=speed
fourcc=cv2.VideoWriter_fourcc(*'mp4v')
video_out=cv2.VideoWriter('bouncing_ball_video.mp4',fourcc,30,(width,height))

In [6]:
def draw_frame(frame,ball_x,ball_y,ball_radious,ball_color):
    frame[:]= background_image
    cv2.circle(frame,(int(ball_x),int(ball_y)),ball_radious,ball_color,-1)

def check_collision(ball_x,ball_y):
    search_radius = ball_radius +1
    search_region = gray_background[int(ball_y) - search_radius:int (ball_y)+search_radius+1,int(ball_x)+search_radius+1
    ]
    return np.any(search_region< 50)

    

cv2.imread loads the background image from the file "background_image.PNG".
cv2.cvtColor converts the background image to grayscale, though the grayscale version is not used later in the script.
The dimensions (height and width) of the background image are extracted to set the frame size for the video.

In [7]:
num_frames = 30 * 60  # 30 frames per second for 60 seconds

for i in range(num_frames):
    frame = np.zeros((height, width, 3), dtype=np.uint8) #Creates a blank frame of the same dimensions as the background image, initialized to black (all zeros).
    ball_x += velocity_x #Updates the ball's horizontal position based on its current velocity.
    ball_y += velocity_y #Updates the ball's vertical position based on its current velocity.

    if check_collision(ball_x, ball_y): #Checks for a collision at the ball's current position. The collision detection logic is not defined here but is intended to return a boolean value.
        pixel_color = None
        if velocity_x > 0:
            pixel_color = background_image[int(ball_y), int(ball_x) + ball_radius]
        elif velocity_x < 0:
            pixel_color = background_image[int(ball_y), int(ball_x) - ball_radius]

        if np.all(pixel_color < 50): #checks if the pixel color is very dark (almost black), which is considered a collision.
            velocity_x *= -1

        if velocity_y > 0:
            pixel_color = background_image[int(ball_y) + ball_radius, int(ball_x)]
        elif velocity_y < 0:
            pixel_color = background_image[int(ball_y) - ball_radius, int(ball_x)]

        if np.all(pixel_color < 50):
            velocity_y *= -1

    if ball_x - ball_radius < 0: 
        ball_x = ball_radius
        velocity_x *= -1
    elif ball_x + ball_radius > width:
        ball_x = width - ball_radius - 1
        velocity_x *= -1

    if ball_y - ball_radius < 0:
        ball_y = ball_radius
        velocity_y *= -1
    elif ball_y + ball_radius > height:
        ball_y = height - ball_radius - 1
        velocity_y *= -1
#These conditions check if the ball has hit the frame boundaries (left, right, top, bottom).
#If the ball hits a boundary, it resets the ball's position to stay within the frame 
#and reverses its velocity to simulate a bounce.

    draw_frame(frame, ball_x, ball_y, ball_radius, ball_color) #Draws the ball on the current frame at the updated position.
    video_out.write(frame) #Writes the current frame to the video file.

video_out.release() #Releases the video writer object, finalizing and saving the video file.